In [1]:
# !pip install gymnasium renderlab
# !pip install opencv-python
# !pip install pygame

In [2]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [25]:
#visualise maze:
rfpMaze = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
maze1 = ["SFFH", "FHHF", "FHFG", "FFFH", "HFHH"]

desc = rfpMaze
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

18
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]


In [26]:
# Q-LEARNING FUNCTIONS

epsilonValue = 0.25
alpha = 0.6
gamma = 0.7
qTable_1 = {}
currentState = 0

def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    if changeInDist > 0:
        return changeInDist/2
    else:
        return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

convergenceThresh = 0.001
def updateTable(direction, nextState, reward):
    global qTable_1
    didConverge = False
    global currentState
    updated = qTable_1[currentState][direction] + alpha*(reward + (gamma*max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and pathFound():
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ

In [27]:
maxEpisodes = 1000
currentEpisode = 1
converged = False
revistPenalty = -0.25

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0
resetVisited()

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    global currentSteps
    if converged:
        break
        
    direction = nextStep(currentState)
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
        if timesVisited[nextState] > 0:
            reward += revistPenalty*timesVisited[nextState]
    
    converged, changeInQ = updateTable(direction, nextState, reward)
    

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1
            resetVisited()

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 20/1000
Time: 42.568 sec
Q-Table:
0: [0, 1.086014655603021, 0.10462923904512986, 0]
1: [0.07216352378580493, -6.0, -0.8323401986031435, 0]
2: [-0.7431872959022905, -8.4, -0.7224543437934811, 0]
3: [-1.179840486442572, -9.36, 0, 0]
4: [0, 1.0331416661131887, -8.4, -0.47597298621923245]
5: [0, -0.26596416917736576, -0.5764967792695687, -0.7243460267957496]
6: [0, -0.3633307652783935, 0, -0.6060137421122268]
7: [-0.7774949947792743, -0.26596416917736576, 0, -0.4330615706616839]
8: [0, -8.4, 0.9291757040345829, 0]
9: [0, 0.9250761488734145, 0.07082039324993694, 0]
10: [0.10755771848718566, 0, 0, 0]
11: [0, 0, 0, 0]
12: [0, -0.5746664644738012, -1.1322225888707225, -1.0233854148734205]
13: [-8.4, 0.8435659741865001, 0, -0.21990278279566972]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0, 0]
17: [0, 0, 0.99934464, -0.17322523570063147]
18: [-3.3297755502473896, 0, -3.4634087255188346, -3.566065981885371]
19: [0, 0, 0, 0]
change in Q: 0.00098
42.567 seconds to converge


In [ ]:
env.reset()

In [ ]:
env.close()